<a name="top"></a><img src="./source/SpinalHDL.png" alt="SpinalHDL based on Scala" style="width:320px;" />

Before running Spinal HDL code, be sure to load SpinalHDL Libraries  
**Note** : This may be a little slow when the first time load, please wait a moment to download Lib from remote.)   

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-spinal.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

## Assignement overlap
SpinalHDL will check that no signal assignement completly erases a previous one.   
The following code will throw the following error:

In [ ]:
class TopLevel extends Component {
  val a = UInt(8 bits)
  a := 42
  a := 66 //Erease the a := 42 :(
}
showRtl(new TopLevel)

A fix could be:

In [ ]:
class TopLevel extends Component {
  val a = UInt(8 bits)
  a := 42
  when(something){
    a := 66
  }
}
showRtl(new TopLevel)

But in the case you really want to override the previous assignement (Yes, it could make sense in some cases), you can do the following:

In [ ]:
class TopLevel extends Component {
  val a = UInt(8 bits)
  a := 42
  a.allowOverride
  a := 66
}
showRtl(new TopLevel)

## Clock crossing violation
SpinalHDL will check that every register of your design only depends (through some combinatorial logic) on registers which use the same or a synchronous clock domain. 

The following code will throw error:

In [ ]:
class TopLevel extends Component {
  val clkA = ClockDomain.external("clkA")
  val clkB = ClockDomain.external("clkB")

  val regA = clkA(Reg(UInt(8 bits)))   //PlayDev.scala:834
  val regB = clkB(Reg(UInt(8 bits)))   //PlayDev.scala:835

  val tmp = regA + regA                //PlayDev.scala:838
  regB := tmp
}
showRtl(new TopLevel)

There are multiple possible fixes:
### crossClockDomain tag
The crossClockDomain tag can be used to say “It’s alright, don’t panic” to SpinalHDL

In [ ]:
class TopLevel extends Component {
  val clkA = ClockDomain.external("clkA")
  val clkB = ClockDomain.external("clkB")

  val regA = clkA(Reg(UInt(8 bits)))
  val regB = clkB(Reg(UInt(8 bits))).addTag(crossClockDomain)


  val tmp = regA + regA
  regB := tmp
}
showRtl(new TopLevel)

### setSyncronousWith
You can also specify that two clock domains are syncronous together.

In [ ]:
class TopLevel extends Component {
  val clkA = ClockDomain.external("clkA")
  val clkB = ClockDomain.external("clkB")
  clkB.setSyncronousWith(clkA)

  val regA = clkA(Reg(UInt(8 bits)))
  val regB = clkB(Reg(UInt(8 bits)))


  val tmp = regA + regA
  regB := tmp
}

showRtl(new TopLevel)

### BufferCC
Signal Bits or Gray-coded Bits can use BufferCC to cross different clockDomain

In [ ]:
class syncRead2Write extends Component {
  val io = new Bundle{
    val pushClock, pushRst = in Bool()
    val readPtr = in UInt(8 bits)
  }
  val pushCC = new ClockingArea(ClockDomain(io.pushClock, io.pushRst)) {
    val pushPtrGray = RegNext(toGray(io.readPtr)) init(0)
  }
}
showRtl(new syncRead2Write)

## Combinational loop
SpinalHDL will check that there are no combinatorial loops in the design.

The following code will throw error:

In [ ]:
class TopLevel extends Component {
  val a = UInt(8 bits) //PlayDev.scala line 831
  val b = UInt(8 bits) //PlayDev.scala line 832
  val c = UInt(8 bits)
  val d = UInt(8 bits)

  a := b
  b := c | d
  d := a
  c := 0
}
showRtl(new TopLevel)

A possible fix could be:

In [ ]:
class TopLevel extends Component {
  val a = UInt(8 bits) //PlayDev.scala line 831
  val b = UInt(8 bits) //PlayDev.scala line 832
  val c = UInt(8 bits)
  val d = UInt(8 bits)

  a := b
  b := c | d
  d := 42
  c := 0
}
showRtl(new TopLevel)

### False-positive
It should be said that SpinalHDL’s algorithm to detect combinatorial loops can be pessimistic, and it may give false positives. If it is giving a false positive, you can manualy disable loop checking on one signal of the loop like so:

In [ ]:
class TopLevel extends Component {
  val a = UInt(8 bits)
  a := 0
  a(1) := a(0) //False positive because of this line
}
showRtl(new TopLevel)

could be fixed by :

In [ ]:
class TopLevel extends Component {
  val a = UInt(8 bits).noCombLoopCheck
  a := 0
  a(1) := a(0)
}
showRtl(new TopLevel)

It should also be said that assignments such as (a(1) := a(0)) can make some tools like Verilator unhappy. It may be better to use a Vec(Bool, 8) in this case

## Hierarchy violation

SpinalHDL will check that signals are never accessed outside of the current component’s scope.

The following signals can be read inside a component:

- All directionless signals defined in the current component

- All in/out/inout signals of the current component

- All in/out/inout signals of children components

In addition, the following signals can be assigned to inside a component:

- All directionless signals defined in the current component

- All out/inout signals of the current component

- All in/inout signals of children components

If a `HIERARCHY VIOLATION`error appears, it means that one of the above rules was violated.

The following code will throw error:

In [ ]:
class TopLevel extends Component {
  val io = new Bundle {
    val a = in UInt(8 bits)
  }
  val tmp = U"x42"
  io.a := tmp
}
showRtl(new TopLevel)

A fix could be :

In [ ]:
class TopLevel extends Component {
  val io = new Bundle {
    val a = out UInt(8 bits) // changed from in to out
  }
  val tmp = U"x42"
  io.a := tmp
}
showRtl(new TopLevel)

## Io bundle
SpinalHDL will check that each io bundle contains only in/out/inout signals.

In [ ]:
class TopLevel extends Component {
  val io = new Bundle {
    val a = UInt(8 bits)
  }
}
showRtl(new TopLevel)

A fix could be:

In [ ]:
class TopLevel extends Component {
  val io = new Bundle {
    val a = in UInt(8 bits)
  }
}
showRtl(new TopLevel)

But if for meta hardware description reasons you realy want io.a to be directionless, you can do:

In [ ]:
class TopLevel extends Component {
  val io = new Bundle {
    val a = UInt(8 bits)
  }
  io.a.allowDirectionLessIo
}
showRtl(new TopLevel)

## Latch detected
SpinalHDL will check that no combinatorial signal will infer a latch during synthesis. In other words, that no combinatorial signals are partialy assigned.

In [ ]:
class TopLevel extends Component {
  val cond = in(Bool)
  val a = UInt(8 bits)

  when(cond){
    a := 42
  }
}
showRtl(new TopLevel)

A fix could be:

In [ ]:
class TopLevel extends Component {
  val cond = in(Bool)
  val a = UInt(8 bits)

  a := 0
  when(cond){
    a := 42
  }
}
showRtl(new TopLevel)

## No driver on
SpinalHDL will check that all combinatorial signals which have an impact on the design are assigned by something.

In [ ]:
class TopLevel extends Component {
  val result = out(UInt(8 bits))
  val a = UInt(8 bits)
  result := a
}
showRtl(new TopLevel)

A fix could be:

In [ ]:
class TopLevel extends Component {
  val result = out(UInt(8 bits))
  val a = UInt(8 bits)
  a := 42
  result := a
}
showRtl(new TopLevel)

## NullPointerException
NullPointerException is a Scala runtime reported error which can happen when a variable is accessed before it was initialised.

In [ ]:
class TopLevel extends Component {
  a := 42
  val a = UInt(8 bits)
}
showRtl(new TopLevel)

A fix could be:

In [ ]:
class TopLevel extends Component {
  val a = UInt(8 bits)
  a := 42
}
showRtl(new TopLevel)

**Issue explanation :**
SpinalHDL is not a language, it is a Scala library, which means it obeys the same rules as the Scala general purpose programming language. When you run your SpinalHDL hardware description to generate the corresponding VHDL/Verilog RTL, your SpinalHDL hardware description will be executed as a Scala programm, and a will be a null reference until the program executes val a = UInt(8 bits), so trying to assign it before then will result in a NullPointerException.

## Register defined as component input
In SpinalHDL, you are not allowed to define a component that has a register as an input. The reasoning behind this is to prevent surprises when the user tries to drive the inputs of child components with the registered signal. If a registered input is desired, you will need to declare the unregistered input in the io bundle, and register the signal in the body of the component.

In [ ]:
class TopLevel extends Component {
  val io = new Bundle {
    val a = in(Reg(UInt(8 bits)))
  }
}
showRtl(new TopLevel)

In [ ]:
class TopLevel extends Component {
  val io = new Bundle {
    val a = in UInt(8 bits)
  }
}
showRtl(new TopLevel)

If a registered a is really wanted, it can be done like so:

In [ ]:
class TopLevel extends Component {
  val io = new Bundle {
    val a = in UInt(8 bits)
  }
  val a = RegNext(io.a)
}
showRtl(new TopLevel)

## Scope violation
SpinalHDL will check that there are no signals assigned outside the scope they are defined in. This error isn’t easy to trigger as it requires some specific meta hardware description tricks.

In [ ]:
class TopLevel extends Component {
  val cond = Bool()

  var tmp : UInt = null
  when(cond){
    tmp = UInt(8 bits)
  }
  tmp := U"x42"
}
showRtl(new TopLevel)

A fix could be:

In [ ]:
class TopLevel extends Component {
  val cond = Bool()

  var tmp : UInt = UInt(8 bits)
  when(cond){

  }
  tmp := U"x42"
}
showRtl(new TopLevel)

## Spinal can’t clone class  ???
This error happens when SpinalHDL wants to create a new datatype via the cloneOf function but isn’t able to do it. The reasons for this is nearly always because it can’t retreive the construction parameters of a Bundle.

In [ ]:
//cloneOf(this) isn't able to retreive the width value that was used to construct itself
class RGB(width : Int) extends Bundle{
  val r,g,b = UInt(width bits)
}
class TopLevel extends Component {
  val tmp =  Stream(new RGB(8))  //Stream requires the capability to cloneOf(new RGB(8))
}
showRtl(new TopLevel)

A fix could be:

In [ ]:
case class RGB(width : Int) extends Bundle{
  val r,g,b = UInt(width bits)
}

class TopLevel extends Component {
  val tmp = Stream(RGB(8))
}
showRtl(new TopLevel)

## Unassigned register
SpinalHDL will check that all registers which impact the design have been assigned somewhere.

In [ ]:
class TopLevel extends Component {
  val result = out(UInt(8 bits))
  val a = Reg(UInt(8 bits))
  result := a
}
showRtl(new TopLevel)

A fix could be:

In [ ]:
class TopLevel extends Component {
  val result = out(UInt(8 bits))
  val a = Reg(UInt(8 bits))
  a := 42
  result := a
}
showRtl(new TopLevel)

### Register with only init  
In some case, because of the design parameterization, it could make sense to generate a register which has no assignement but only a init statement.

In [ ]:
class TopLevel(something: Boolean) extends Component {
  val result = out(UInt(8 bits))
  val a = Reg(UInt(8 bits))  
   
  if(something){ 
    a := 32
  }
  result := a
}
showRtl(new TopLevel(false) )

To fix it you can ask SpinalHDL to transform the register into a combinatorial one if no assignement is present but it as a init statement:

In [ ]:
class TopLevel(something: Boolean) extends Component {
  val result = out(UInt(8 bits))
  val a = Reg(UInt(8 bits)).init(42).allowUnsetRegToAvoidLatch

  if(something){   
    a := 37   
  }
  result := a
}
showRtl(new TopLevel(false))

## Unreachable is statement
SpinalHDL will check all is statements in a switch are reachable.

In [ ]:
class TopLevel extends Component {
  val sel = UInt(2 bits)
  val result = UInt(4 bits)
  switch(sel){
    is(0){ result := 4 }
    is(1){ result := 6 }
    is(2){ result := 8 }
    is(3){ result := 9 }
    is(0){ result := 2 } //Duplicated statement is statement !
  }
}
showRtl(new TopLevel)

A fix could be:

In [ ]:
class TopLevel extends Component {
  val sel = UInt(2 bits)
  val result = UInt(4 bits)
  switch(sel){
    is(0){ result := 4 }
    is(1){ result := 6 }
    is(2){ result := 8 }
    is(3){ result := 9 }
  }
}
showRtl(new TopLevel)

## Width mismatch
SpinalHDL will check that signals on the left and right side of assignments and operators are the same width.

In [ ]:
class TopLevel extends Component {
  val a = UInt(8 bits)
  val b = UInt(4 bits)
  b := a
}
showRtl(new TopLevel)

A fix could be:

In [ ]:
class TopLevel extends Component {
  val a = UInt(8 bits)
  val b = UInt(4 bits)
  b := a.resized
}
showRtl(new TopLevel)

###  Operator example
The following code:

In [ ]:
class TopLevel extends Component {
  val a = UInt(8 bits)
  val b = UInt(4 bits)
  val result = a | b
}
showRtl(new TopLevel)

A fix could be:

In [ ]:
class TopLevel extends Component {
  val a = UInt(8 bits)
  val b = UInt(4 bits)
  val result = a | (b.resized)
}
showRtl(new TopLevel)